In [ ]:
SANDBOX_NAME = # Sandbox Name
DATA_PATH = "/data/sandboxes/"+SANDBOX_NAME+"/data/"



# Matplotlib - Representación de datos



En este notebook veremos diferentes formas de representar variables con tal de extraer información de los datos a partir de su representación gráfica.

In [ ]:
import pandas as pd
data_raw = spark.read.csv(DATA_PATH+"day.csv", header=True,
                          inferSchema=True).toPandas()

# Local read
# data_raw = pd.read_csv("../../data/Bike-Sharing-Dataset/day.csv")
#https://archive.ics.uci.edu/ml/datasets/dow+jones+index
data_raw.head()



tratamiento de datos de las variables


In [ ]:
data = data_raw.copy()

# convert the variable with information about date to its correct format
data['dteday'] = pd.to_datetime(data['dteday'])

# drop variables that will not be use
drop_list = ['instant', 'season', 'yr', 'mnth', 'holiday', 'workingday', 'weathersit', 'atemp', 'hum']
data.drop(drop_list, inplace = True, axis = 1)

In [ ]:
data.head()



## Relaciones entre variables cuantitativas

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline



### Scatter plot



se quiere hacer gráfica del número de bicicletas utilizadas en función de la temperatura. Un 'scatter plot' proporciona
una representación poco agregada de los datos.

In [ ]:
plt.scatter(data['temp'], data['cnt'], alpha=0.5)
plt.xlabel('Normalized temperature (C)')
plt.ylabel('Used bikes')
plt.title('Used bicycles according to the normalized temperature')
plt.show()



### Plot con 2 ejes Y



Este tipo de gráfica resulta muy útil cuando se quieren comparar dos variables cuantitativas en función de una tercera variable pero las dos tienen escalas muy distintas.



Cada variable tendrá su propio objeto plot pero se representarán en uno solo.

In [ ]:
_,ax1 = plt.subplots()
ax1.plot(data['dteday'], data['cnt'], color = 'b')

ax1.set_ylabel('Used bikes')
ax1.set_xlabel('Day')
ax1.set_title('Used bikes and wind speed over the time')

# We create a second plot object, indicating that both of them have the same  axis.
ax2 = ax1.twinx()
ax2.plot(data['dteday'], data['windspeed'], color = 'g')
ax2.set_ylabel('wind speed')

plt.show()



## Distribución de los datos



### Histograma



Muy útiles para ver la distribución de una variable cuantitative a grandes rasgos.

In [ ]:
plt.hist(data['Registered'])
plt.xlabel('Used bikes')
plt.ylabel('Frequency')
plt.title('Used bikes number distribution')
plt.show()



También se pueden crear dos histogramas en uno con tal de comparar dos distribuciones.



se fijan los bins para que sean los mismos para ambos histogramas

In [ ]:
import numpy as np

max_nbins = 10
data_range = [min(min(data['registered']), min(data['casual'])), max(max(data['registered']), max(data['casual']))]
binwidth = (data_range[1] - data_range[0]) / max_nbins
bins = np.arange(data_range[0], data_range[1] + binwidth, binwidth)

plt.hist(data['registered'], bins=bins, color='b', alpha=1, label='registered')
plt.hist(data['casual'], bins=bins, color='r', alpha=0.75, label = 'casual')

plt.xlabel('Used bikes')
plt.ylabel('Frequency')
plt.title('Used bikes number distribution per type of client')
plt.legend(loc='best')
plt.show()



### Density plot



Método más rigusoso para determinar la forma de la distribución de los datos.



El primer paso es estimar la función de densidad de probabilidad de los datos en cuestión

In [ ]:
from scipy.stats import gaussian_kde

density_est = gaussian_kde(data['registered'])

# Create ordered data
x_data = np.arange(min(data['registered']), max(data['registered']), 200)

plt.plot(x_data, density_est(x_data))
plt.xlabel('Used bikes')
plt.ylabel('Frequency')
plt.title('Used bikes distribution')
plt.show()



## Comparar grupos/categorías



### Diagrama de barras



Para entender mejor el comportamiento de las variables categóricas, resulta útil añadir barras de error. 

In [ ]:
mean_total_co_day = data[['weekday', 'cnt']].groupby('weekday').agg([np.mean, np.std])



Calcular la media y desviación estándard para el número de bicicletas utilizadas cada día

In [ ]:
mean_total_co_day = data[['weekday', 'cnt']].groupby('weekday').agg([np.mean, np.std])
mean_total_co_day.columns = mean_total_co_day.columns.droplevel()

_, ax = plt.subplots()
ax.bar(mean_total_co_day.index.values, mean_total_co_day['mean'], color = 'b', align = 'center')
# Add the errors. The parameter ls set to 'none' so that it does not exist a line between the different bars
ax.errorbar(mean_total_co_day.index.values, mean_total_co_day['mean'],ls= 'none', yerr = mean_total_co_day['std'], color = 'black', capthick = 2)
ax.set_ylabel('day of week')
ax.set_xlabel('bikes used')
ax.set_title('Total used bicycles per day of the week (0 = Sunday)')
plt.show()



### Diagrama de barras apilado



Resultan especialmente útiles para comparar proporciones entre categorías. 



Ejemplo con el número medio de bicicletas utilizadas por día de semana y tipo de cliente

In [ ]:
mean_by_reg_co_day = data[['weekday', 'registered', 'casual']].groupby('weekday').mean()

# Calculate the proportion of each type of customer per day
mean_by_reg_co_day['total'] = mean_by_reg_co_day['registered'] + mean_by_reg_co_day['casual']
mean_by_reg_co_day['reg_prop'] = mean_by_reg_co_day['registered'] / mean_by_reg_co_day['total']
mean_by_reg_co_day['casual_prop'] = mean_by_reg_co_day['casual'] / mean_by_reg_co_day['total']

# Graph values
x_data = mean_by_reg_co_day.index.values
y_data_list = [mean_by_reg_co_day['reg_prop'], mean_by_reg_co_day['casual_prop']]
y_data_names = ['registered', 'casual']
# colors can be indicated in Hex format
colors = ['#539caf', '#7663b0']


_, ax = plt.subplots()
# Bars are plotted one by one in order to stack information
for i in range(0, len(y_data_list)):
    if i == 0:
        ax.bar(x_data, y_data_list[i], color = colors[i], align = 'center', label = y_data_names[i])
    else:
        # Information is piled just above each bar
        ax.bar(x_data, y_data_list[i], color = colors[i], bottom = y_data_list[i - 1], align = 'center', label = y_data_names[i])
ax.set_ylabel('Used bikes proportion')
ax.set_xlabel('Day of the week')
ax.set_title('Used bikes per day of the week (0 = Sunday)')
ax.legend(loc = 'best')
plt.show()



### Diagrama de barras agrupado



Resultan especialmente útiles para comparar valores absolutos entre categorías. 



Ejemplo con el número medio de bicicletas utilizadas por día de semana y tipo de cliente

In [ ]:
# Data to be used
x_data = mean_by_reg_co_day.index.values
y_data_list = [mean_by_reg_co_day['registered'], mean_by_reg_co_day['casual']]
y_data_names = ['registered', 'casual']
# It is also possible to indicate the color in RGB format
colors = [(0.53,0.11,0.33), '#7663b0']

_, ax = plt.subplots()
# Fixed bar width
total_width = 0.8
# Thickness of each of the bars
ind_width = total_width / len(y_data_list)
# Center both bars around the corresponding X axis tick.
alteration = np.arange(-(total_width/2), total_width/2, ind_width)

# Draw bars, one category at a time
for i in range(0, len(y_data_list)):
    # Move the bar to the right on the x-axis so it doesn't
    # overlap with previously drawn ones
    ax.bar(x_data + alteration[i], y_data_list[i], color = colors[i], label = y_data_names[i], width = ind_width)
ax.set_ylabel('Used bikes proportion')
ax.set_xlabel('Day of the week')
ax.set_title('Used bikes per day of the week (0 = Sunday)')
ax.legend(loc = 'Upper right')
plt.show()



### Boxplots



 Especialmente útiles para representar la distribución de una variable sobre distintas categorías



Se tiene que indicar la variable cuyas categorías se usarán en la gráfica y calcular valores para cada categoría

In [ ]:
bp_data = []
for day in days:
    bp_data.append(data[data['weekday'] == day]['cnt'].values)

base_color = '#539caf'
median_color = '#297083'

plt.boxplot(bp_data
           # parameter that allow to plot
           , patch_artist = True
           # properties of the median
           , medianprops = {'color': median_color}
           # properties of the box
           , boxprops = {'color': base_color, 'facecolor': base_color}
           # properties of the whiskers
           , whiskerprops = {'color': base_color}
           # properties of the whisker boundaries
           , capprops = {'color': base_color})

plt.xticks(np.arange(1,len(days)+1), days)
plt.ylabel('Used bikes')
plt.xlabel('Day of the week')
plt.title('Used bikes per day of the week (0 = Sunday)')
plt.show()